In [2]:
import os
import sys
import wandb
import argparse
import numpy as np


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))
import torch
import torchvision.transforms as T
import torchvision

from dataloaders.dataloader_cifar10 import get_cifar10
from dataloaders.dataloader_cifar100 import get_cifar100
from utils.eval_metrics import linear_evaluation, get_t_SNE_plot
from models.linear_classifer import LinearClassifier
from models.ssl import  SimSiam, Siamese, Encoder, Predictor

from trainers.train_simsiam import train_simsiam
from trainers.train_infomax import train_infomax
from trainers.train_barlow import train_barlow

from trainers.train_PFR import train_PFR_simsiam
from trainers.train_PFR_contrastive import train_PFR_contrastive_simsiam
from trainers.train_contrastive import train_contrastive_simsiam
from trainers.train_ering import train_ering_simsiam

from torchsummary import summary
import random
from utils.lr_schedulers import LinearWarmupCosineAnnealingLR, SimSiamScheduler
from utils.eval_metrics import Knn_Validation_cont
from copy import deepcopy
from loss import invariance_loss,CovarianceLoss,ErrorCovarianceLoss
import torch.nn as nn
import time
import torch.nn.functional as F
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"

In [3]:

path = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_0_same_lr_True_norm_batch_ws_False.pth.tar"
ti_c = os.path.getctime(path)
ti_m = os.path.getmtime(path)
# Converting the time in seconds to a timestamp
c_ti = time.ctime(ti_c)
m_ti = time.ctime(ti_m)
print(f"The file was created at {c_ti} and was last modified at {m_ti}")

path = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_1_same_lr_True_norm_batch_ws_False.pth.tar"
ti_c = os.path.getctime(path)
ti_m = os.path.getmtime(path)
# Converting the time in seconds to a timestamp
c_ti = time.ctime(ti_c)
m_ti = time.ctime(ti_m)
print(f"The file was created at {c_ti} and was last modified at {m_ti}")

path = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_2_same_lr_True_norm_batch_ws_False.pth.tar"
ti_c = os.path.getctime(path)
ti_m = os.path.getmtime(path)
# Converting the time in seconds to a timestamp
c_ti = time.ctime(ti_c)
m_ti = time.ctime(ti_m)
print(f"The file was created at {c_ti} and was last modified at {m_ti}")

path = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_3_same_lr_True_norm_batch_ws_False.pth.tar"
ti_c = os.path.getctime(path)
ti_m = os.path.getmtime(path)
# Converting the time in seconds to a timestamp
c_ti = time.ctime(ti_c)
m_ti = time.ctime(ti_m)
print(f"The file was created at {c_ti} and was last modified at {m_ti}")

path = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_4_same_lr_True_norm_batch_ws_False.pth.tar"
ti_c = os.path.getctime(path)
ti_m = os.path.getmtime(path)
# Converting the time in seconds to a timestamp
c_ti = time.ctime(ti_c)
m_ti = time.ctime(ti_m)
print(f"The file was created at {c_ti} and was last modified at {m_ti}")

The file was created at Fri Jul 14 19:23:25 2023 and was last modified at Fri Jul 14 19:23:25 2023
The file was created at Sat Jul 15 04:23:11 2023 and was last modified at Sat Jul 15 04:23:11 2023
The file was created at Sat Jul 15 13:08:18 2023 and was last modified at Sat Jul 15 13:08:18 2023
The file was created at Sat Jul 15 21:09:45 2023 and was last modified at Sat Jul 15 21:09:45 2023
The file was created at Sun Jul 16 05:08:18 2023 and was last modified at Sun Jul 16 05:08:18 2023


In [4]:
class GaussianBlur(object):
    """Gaussian blur augmentation in SimCLR https://arxiv.org/abs/2002.05709"""

    def __init__(self, sigma=[0.1, 2.0]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = torchvision.transforms.functional.gaussian_blur(x,kernel_size=[3,3],sigma=sigma)#kernel size and sigma are open problems but right now seems ok!
        return x


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [5]:
class Args():
    normalization = 'batch'
    weight_standard = False
    same_lr = False
    pretrain_batch_size = 512
    pretrain_warmup_epochs = 10
    pretrain_warmup_lr = 3e-3
    pretrain_base_lr = 0.03
    pretrain_momentum = 0.9
    pretrain_weight_decay = 5e-4
    min_lr = 0.00
    lambdap = 1.0
    appr = 'cassle_barlow'
    knn_report_freq = 10
    cuda_device = 7
    num_workers = 8
    contrastive_ratio = 0.001
    dataset = 'cifar100'
    class_split = [20,20,20,20,20]
    epochs = [500,500,500,500,500]
    cov_loss_weight = 1.0
    sim_loss_weight = 250.0
    info_loss = 'invariance'
    lambda_norm = 1.0
    subspace_rate = 0.99
    lambda_param = 5e-3
    bsize = 32
    msize = 150
    proj_hidden = 2048
    proj_out = 2048 #infomax 64
    pred_hidden = 512
    pred_out = 2048



In [6]:
args = Args()

In [7]:
if args.dataset == "cifar10":
    get_dataloaders = get_cifar10
    num_classes=10
elif args.dataset == "cifar100":
    get_dataloaders = get_cifar100
    num_classes=100
assert sum(args.class_split) == num_classes
assert len(args.class_split) == len(args.epochs)

In [8]:
num_worker = args.num_workers
#device
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)

cuda:7


In [9]:
#wandb init
wandb.init(project="CSSL",  entity="yavuz-team",
            mode="disabled",
            config=args,
            name= str(args.dataset) + '-algo' + str(args.appr) + "-e" + str(args.epochs) + "-b" 
            + str(args.pretrain_batch_size) + "-lr" + str(args.pretrain_base_lr)+"-CS"+str(args.class_split))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [10]:
if 'infomax' in args.appr or 'barlow' in args.appr:
    transform = T.Compose([
            T.RandomResizedCrop(size=32, scale=(0.2, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)]), p=0.8),
            T.RandomGrayscale(p=0.2),
            T.RandomApply([GaussianBlur()], p=0.5), 
            T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

    transform_prime = T.Compose([
            T.RandomResizedCrop(size=32, scale=(0.2, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)]), p=0.8),
            T.RandomGrayscale(p=0.2),
            T.RandomApply([GaussianBlur()], p=0.5), 
            T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

In [11]:
#Dataloaders
print("Creating Dataloaders..")
#Class Based
train_data_loaders, train_data_loaders_knn, test_data_loaders, _, train_data_loaders_linear, train_data_loaders_pure  = get_dataloaders(transform, transform_prime, \
                                    classes=args.class_split, valid_rate = 0.00, batch_size=args.pretrain_batch_size, seed = 0, num_worker= num_worker)
_, train_data_loaders_knn_all, test_data_loaders_all, _, train_data_loaders_linear_all, train_data_loaders_pure_all = get_dataloaders(transform, transform_prime, \
                                        classes=[num_classes], valid_rate = 0.00, batch_size=args.pretrain_batch_size, seed = 0, num_worker= num_worker)


Creating Dataloaders..
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [12]:

from tqdm import tqdm
def correct_top_k(outputs, targets, top_k=(1,5)):
    with torch.no_grad():
        prediction = torch.argsort(outputs, dim=-1, descending=True)
        result= []
        for k in top_k:
            correct_k = torch.sum((prediction[:, 0:k] == targets.unsqueeze(dim=-1)).any(dim=-1).float()).item() 
            result.append(correct_k)
        return result

def linear_test(net, data_loader, classifier, epoch, device, task_sep=False, intra_task=False):
    # evaluate model:
    net.eval() # for not update batchnorm
    linear_loss = 0.0
    num = 0
    total_loss, total_correct_1, total_correct_5, total_num, test_bar = 0.0, 0.0, 0.0, 0, tqdm(data_loader)
    with torch.no_grad():
        for data_tuple in test_bar:
            data, target = [t.to(device) for t in data_tuple]
            if task_sep:
                target = target // 20
            if intra_task:
                target = target % 20
            output = net(data)
            if classifier is not None:  #else net is already a classifier
                output = classifier(output) 
            linear_loss = F.cross_entropy(output, target)
            
            # Batchsize for loss and accuracy
            num = data.size(0)
            total_num += num 
            total_loss += linear_loss.item() * num 
            # Accumulating number of correct predictions 
            correct_top_1, correct_top_5 = correct_top_k(output, target, top_k=(1,5))    
            total_correct_1 += correct_top_1
            total_correct_5 += correct_top_5

            test_bar.set_description('Lin.Test Epoch: [{}] Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}% '
                                     .format(epoch,  total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_5 / total_num * 100
                                             ))
        acc_1 = total_correct_1/total_num*100
        acc_5 = total_correct_5/total_num*100
        wandb.log({" Linear Layer Test Loss ": linear_loss / total_num, " Epoch ": epoch})
        wandb.log({" Linear Layer Test - Acc": acc_1, " Epoch ": epoch})
    return total_loss / total_num, acc_1 , acc_5 

def linear_train(net, data_loader, train_optimizer, classifier, scheduler, epoch, device, task_sep=False, intra_task=False):

    net.eval() # for not update batchnorm 
    total_num, train_bar = 0, tqdm(data_loader)
    linear_loss = 0.0
    total_correct_1, total_correct_5 = 0.0, 0.0
    for data_tuple in train_bar:
        # Forward prop of the model with single augmented batch
        pos_1, target = data_tuple
        pos_1 = pos_1.to(device)
        feature_1 = net(pos_1)
        # feature_1 = net.get_representation(pos_1) 

        # Batchsize
        batchsize_bc = feature_1.shape[0]
        features = feature_1
        targets = target.to(device)
        if task_sep:
            targets = targets // 20
        if intra_task:
            targets = targets % 20
        logits = classifier(features.detach()) 
        # Cross Entropy Loss 
        linear_loss_1 = F.cross_entropy(logits, targets)

        # Number of correct predictions
        linear_correct_1, linear_correct_5 = correct_top_k(logits, targets, top_k=(1, 5))
    

        # Backpropagation part
        train_optimizer.zero_grad()
        linear_loss_1.backward()
        train_optimizer.step()

        # Accumulating number of examples, losses and correct predictions
        total_num += batchsize_bc
        linear_loss += linear_loss_1.item() * batchsize_bc
        total_correct_1 += linear_correct_1 
        total_correct_5 += linear_correct_5


        acc_1 = total_correct_1/total_num*100
        # # This bar is used for live tracking on command line (batch_size -> batchsize_bc: to show current batchsize )
        train_bar.set_description('Lin.Train Epoch: [{}] Loss: {:.4f} ACC: {:.2f}'.format(\
                epoch, linear_loss / total_num, acc_1))
    scheduler.step()
    acc_1 = total_correct_1/total_num*100
    acc_5 = total_correct_5/total_num*100       
    wandb.log({" Linear Layer Train Loss ": linear_loss / total_num, " Epoch ": epoch})
    wandb.log({" Linear Layer Train - Acc": acc_1, " Epoch ": epoch})
    # print(f'Linear Layer Train - Acc: {acc_1}')
        
    return linear_loss/total_num, acc_1, acc_5


def linear_evaluation(net, data_loader,test_data_loader,train_optimizer,classifier, scheduler, epochs, device, task_sep=False, intra_task=False):
    for epoch in range(1, epochs+1):
        linear_loss, linear_acc1, linear_acc5 = linear_train(net,data_loader,train_optimizer,classifier,scheduler, epoch, device, task_sep, intra_task)
        with torch.no_grad():
            # Testing for linear evaluation
            test_loss, test_acc1, test_acc5 = linear_test(net, test_data_loader, classifier, epoch, device, task_sep, intra_task)

    return test_loss, test_acc1, test_acc5, classifier

### Task 0 Model

In [13]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard, appr_name = args.appr)
    model = Siamese(encoder)
    model.to(device) #automatically detects from model

cuda:7


In [14]:
#load model here
file_name = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_0_same_lr_True_norm_batch_ws_False.pth.tar"
dict = torch.load(file_name)

In [15]:
model.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)

In [16]:
model.load_state_dict(dict['state_dict'])

<All keys matched successfully>

In [17]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 100).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 3.5335 ACC: 25.97: 100%|██████████| 196/196 [00:13<00:00, 14.98it/s]
Lin.Test Epoch: [1] Loss: 3.2774 ACC@1: 31.53% ACC@5: 61.87% : 100%|██████████| 20/20 [00:03<00:00,  6.22it/s]
Lin.Train Epoch: [2] Loss: 3.3858 ACC: 30.61: 100%|██████████| 196/196 [00:11<00:00, 16.61it/s]
Lin.Test Epoch: [2] Loss: 3.2100 ACC@1: 34.18% ACC@5: 64.50% : 100%|██████████| 20/20 [00:03<00:00,  6.43it/s]
Lin.Train Epoch: [3] Loss: 3.3244 ACC: 32.02: 100%|██████████| 196/196 [00:11<00:00, 16.71it/s]
Lin.Test Epoch: [3] Loss: 2.8828 ACC@1: 37.58% ACC@5: 67.20% : 100%|██████████| 20/20 [00:03<00:00,  5.91it/s]
Lin.Train Epoch: [4] Loss: 3.2872 ACC: 32.82: 100%|██████████| 196/196 [00:11<00:00, 16.68it/s]
Lin.Test Epoch: [4] Loss: 2.9457 ACC@1: 38.34% ACC@5: 67.42% : 100%|██████████| 20/20 [00:02<00:00,  7.69it/s]
Lin.Train Epoch: [5] Loss: 3.1951 ACC: 33.48: 100%|██████████| 196/196 [00:11<00:00, 16.67it/s]
Lin.Test Epoch: [5] Loss: 2.9307 ACC@1: 37.34% ACC@5: 68.14% : 100%|████████

Total Accuracy: 51.01


In [20]:
task = 0
loader = train_data_loaders_linear[task]
lin_epoch=50
classifier = LinearClassifier(num_classes = 20).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, loader,test_data_loaders[task],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, intra_task=True) 

Lin.Train Epoch: [1] Loss: 3.4219 ACC: 45.87: 100%|██████████| 40/40 [00:03<00:00, 12.84it/s]
Lin.Test Epoch: [1] Loss: 2.8852 ACC@1: 58.90% ACC@5: 88.15% : 100%|██████████| 4/4 [00:01<00:00,  2.82it/s]
Lin.Train Epoch: [2] Loss: 2.6656 ACC: 54.84: 100%|██████████| 40/40 [00:03<00:00, 12.28it/s]
Lin.Test Epoch: [2] Loss: 2.6858 ACC@1: 53.75% ACC@5: 89.25% : 100%|██████████| 4/4 [00:01<00:00,  2.57it/s]
Lin.Train Epoch: [3] Loss: 2.4339 ACC: 55.50: 100%|██████████| 40/40 [00:03<00:00, 12.20it/s]
Lin.Test Epoch: [3] Loss: 2.5195 ACC@1: 56.95% ACC@5: 88.75% : 100%|██████████| 4/4 [00:01<00:00,  2.58it/s]
Lin.Train Epoch: [4] Loss: 2.6829 ACC: 55.66: 100%|██████████| 40/40 [00:03<00:00, 12.91it/s]
Lin.Test Epoch: [4] Loss: 2.1839 ACC@1: 58.95% ACC@5: 92.15% : 100%|██████████| 4/4 [00:01<00:00,  2.48it/s]
Lin.Train Epoch: [5] Loss: 2.2981 ACC: 58.21: 100%|██████████| 40/40 [00:03<00:00, 12.82it/s]
Lin.Test Epoch: [5] Loss: 2.8239 ACC@1: 56.75% ACC@5: 86.15% : 100%|██████████| 4/4 [00:01<00:

In [21]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 5).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, task_sep=True) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 15.2180 ACC: 28.90: 100%|██████████| 196/196 [00:11<00:00, 16.93it/s]
Lin.Test Epoch: [1] Loss: 8.2385 ACC@1: 32.45% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.31it/s]
Lin.Train Epoch: [2] Loss: 11.8133 ACC: 30.38: 100%|██████████| 196/196 [00:12<00:00, 16.18it/s]
Lin.Test Epoch: [2] Loss: 12.2366 ACC@1: 34.26% ACC@5: 100.00% : 100%|██████████| 20/20 [00:02<00:00,  6.73it/s]
Lin.Train Epoch: [3] Loss: 12.3819 ACC: 30.46: 100%|██████████| 196/196 [00:11<00:00, 16.40it/s]
Lin.Test Epoch: [3] Loss: 8.0494 ACC@1: 33.03% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.52it/s]
Lin.Train Epoch: [4] Loss: 11.0473 ACC: 31.56: 100%|██████████| 196/196 [00:11<00:00, 17.72it/s]
Lin.Test Epoch: [4] Loss: 7.6027 ACC@1: 37.60% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.25it/s]
Lin.Train Epoch: [5] Loss: 12.3760 ACC: 30.69: 100%|██████████| 196/196 [00:12<00:00, 16.33it/s]
Lin.Test Epoch: [5] Loss: 6.9354 ACC@1: 37.46% ACC@5: 100.00% : 10

Total Accuracy: 46.17


### Task 1 Model

In [22]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard, appr_name = args.appr)
    model = Siamese(encoder)
    model.to(device) #automatically detects from model

#load model here
file_name = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_1_same_lr_True_norm_batch_ws_False.pth.tar"
dict = torch.load(file_name)
model.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)
model.load_state_dict(dict['state_dict'])

cuda:7


<All keys matched successfully>

In [23]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 100).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 3.9272 ACC: 29.55: 100%|██████████| 196/196 [00:10<00:00, 18.03it/s]
Lin.Test Epoch: [1] Loss: 3.8110 ACC@1: 35.04% ACC@5: 62.97% : 100%|██████████| 20/20 [00:03<00:00,  6.57it/s]
Lin.Train Epoch: [2] Loss: 3.8725 ACC: 34.65: 100%|██████████| 196/196 [00:11<00:00, 16.50it/s]
Lin.Test Epoch: [2] Loss: 3.5389 ACC@1: 38.81% ACC@5: 67.05% : 100%|██████████| 20/20 [00:03<00:00,  6.21it/s]
Lin.Train Epoch: [3] Loss: 3.7951 ACC: 36.24: 100%|██████████| 196/196 [00:11<00:00, 16.74it/s]
Lin.Test Epoch: [3] Loss: 3.3260 ACC@1: 40.50% ACC@5: 69.92% : 100%|██████████| 20/20 [00:03<00:00,  6.09it/s]
Lin.Train Epoch: [4] Loss: 3.7773 ACC: 37.18: 100%|██████████| 196/196 [00:11<00:00, 16.85it/s]
Lin.Test Epoch: [4] Loss: 3.1523 ACC@1: 42.68% ACC@5: 71.28% : 100%|██████████| 20/20 [00:03<00:00,  6.52it/s]
Lin.Train Epoch: [5] Loss: 3.6626 ACC: 37.44: 100%|██████████| 196/196 [00:10<00:00, 18.17it/s]
Lin.Test Epoch: [5] Loss: 3.2202 ACC@1: 42.43% ACC@5: 72.14% : 100%|████████

Total Accuracy: 55.1


In [24]:
task = 1
loader = train_data_loaders_linear[task]
lin_epoch=50
classifier = LinearClassifier(num_classes = 20).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, loader,test_data_loaders[task],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, intra_task=True) 

Lin.Train Epoch: [1] Loss: 3.8118 ACC: 54.89: 100%|██████████| 40/40 [00:03<00:00, 13.00it/s]
Lin.Test Epoch: [1] Loss: 2.0705 ACC@1: 71.30% ACC@5: 92.45% : 100%|██████████| 4/4 [00:01<00:00,  2.57it/s]
Lin.Train Epoch: [2] Loss: 2.5601 ACC: 65.82: 100%|██████████| 40/40 [00:03<00:00, 13.22it/s]
Lin.Test Epoch: [2] Loss: 2.1626 ACC@1: 69.65% ACC@5: 92.70% : 100%|██████████| 4/4 [00:01<00:00,  2.58it/s]
Lin.Train Epoch: [3] Loss: 2.7646 ACC: 65.51: 100%|██████████| 40/40 [00:03<00:00, 11.86it/s]
Lin.Test Epoch: [3] Loss: 2.1942 ACC@1: 69.25% ACC@5: 93.50% : 100%|██████████| 4/4 [00:01<00:00,  3.01it/s]
Lin.Train Epoch: [4] Loss: 2.2519 ACC: 67.25: 100%|██████████| 40/40 [00:02<00:00, 13.45it/s]
Lin.Test Epoch: [4] Loss: 1.9596 ACC@1: 71.95% ACC@5: 93.25% : 100%|██████████| 4/4 [00:01<00:00,  2.51it/s]
Lin.Train Epoch: [5] Loss: 2.1725 ACC: 68.86: 100%|██████████| 40/40 [00:03<00:00, 11.75it/s]
Lin.Test Epoch: [5] Loss: 1.8335 ACC@1: 72.00% ACC@5: 93.35% : 100%|██████████| 4/4 [00:01<00:

In [25]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 5).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, task_sep=True) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 18.0632 ACC: 31.40: 100%|██████████| 196/196 [00:10<00:00, 18.07it/s]
Lin.Test Epoch: [1] Loss: 9.9773 ACC@1: 36.46% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.57it/s] 
Lin.Train Epoch: [2] Loss: 11.7705 ACC: 33.17: 100%|██████████| 196/196 [00:11<00:00, 16.48it/s]
Lin.Test Epoch: [2] Loss: 23.6230 ACC@1: 31.54% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.53it/s]
Lin.Train Epoch: [3] Loss: 13.3592 ACC: 34.05: 100%|██████████| 196/196 [00:12<00:00, 16.14it/s]
Lin.Test Epoch: [3] Loss: 11.3432 ACC@1: 33.76% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  5.79it/s]
Lin.Train Epoch: [4] Loss: 11.8659 ACC: 33.72: 100%|██████████| 196/196 [00:12<00:00, 16.32it/s]
Lin.Test Epoch: [4] Loss: 19.6567 ACC@1: 29.49% ACC@5: 100.00% : 100%|██████████| 20/20 [00:02<00:00,  8.26it/s]
Lin.Train Epoch: [5] Loss: 13.6698 ACC: 33.55: 100%|██████████| 196/196 [00:11<00:00, 16.70it/s]
Lin.Test Epoch: [5] Loss: 7.9209 ACC@1: 36.98% ACC@5: 100.00% :

Total Accuracy: 48.25


### Task 2 Model

In [13]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard, appr_name = args.appr)
    model = Siamese(encoder)
    model.to(device) #automatically detects from model

#load model here
file_name = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_2_same_lr_True_norm_batch_ws_False.pth.tar"
dict = torch.load(file_name)
model.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)
model.load_state_dict(dict['state_dict'])

cuda:7


<All keys matched successfully>

In [28]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 100).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 4.3866 ACC: 31.65: 100%|██████████| 196/196 [00:11<00:00, 16.45it/s]
Lin.Test Epoch: [1] Loss: 3.9141 ACC@1: 39.43% ACC@5: 69.17% : 100%|██████████| 20/20 [00:02<00:00,  6.72it/s]
Lin.Train Epoch: [2] Loss: 4.3498 ACC: 37.20: 100%|██████████| 196/196 [00:11<00:00, 16.50it/s]
Lin.Test Epoch: [2] Loss: 3.7598 ACC@1: 42.63% ACC@5: 71.69% : 100%|██████████| 20/20 [00:03<00:00,  6.10it/s]
Lin.Train Epoch: [3] Loss: 4.4017 ACC: 38.39: 100%|██████████| 196/196 [00:11<00:00, 16.34it/s]
Lin.Test Epoch: [3] Loss: 3.6787 ACC@1: 43.35% ACC@5: 72.89% : 100%|██████████| 20/20 [00:03<00:00,  6.37it/s]
Lin.Train Epoch: [4] Loss: 4.3992 ACC: 39.15: 100%|██████████| 196/196 [00:10<00:00, 17.93it/s]
Lin.Test Epoch: [4] Loss: 3.7347 ACC@1: 43.06% ACC@5: 72.70% : 100%|██████████| 20/20 [00:03<00:00,  6.63it/s]
Lin.Train Epoch: [5] Loss: 4.3703 ACC: 39.95: 100%|██████████| 196/196 [00:11<00:00, 16.49it/s]
Lin.Test Epoch: [5] Loss: 3.8885 ACC@1: 43.99% ACC@5: 73.21% : 100%|████████

Total Accuracy: 57.589999999999996


In [27]:
task = 2
loader = train_data_loaders_linear[task]
lin_epoch=50
classifier = LinearClassifier(num_classes = 20).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, loader,test_data_loaders[task],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, intra_task=True) 

Lin.Train Epoch: [1] Loss: 3.6720 ACC: 55.29: 100%|██████████| 40/40 [00:03<00:00, 11.31it/s]
Lin.Test Epoch: [1] Loss: 3.9104 ACC@1: 59.45% ACC@5: 91.70% : 100%|██████████| 4/4 [00:01<00:00,  2.93it/s]
Lin.Train Epoch: [2] Loss: 2.8925 ACC: 66.20: 100%|██████████| 40/40 [00:03<00:00, 11.74it/s]
Lin.Test Epoch: [2] Loss: 2.8489 ACC@1: 63.90% ACC@5: 93.20% : 100%|██████████| 4/4 [00:01<00:00,  2.36it/s]
Lin.Train Epoch: [3] Loss: 3.0994 ACC: 66.18: 100%|██████████| 40/40 [00:03<00:00, 12.44it/s]
Lin.Test Epoch: [3] Loss: 2.4839 ACC@1: 70.00% ACC@5: 93.15% : 100%|██████████| 4/4 [00:01<00:00,  2.60it/s]
Lin.Train Epoch: [4] Loss: 2.9922 ACC: 67.48: 100%|██████████| 40/40 [00:03<00:00, 11.49it/s]
Lin.Test Epoch: [4] Loss: 2.9413 ACC@1: 66.05% ACC@5: 93.20% : 100%|██████████| 4/4 [00:01<00:00,  2.51it/s]
Lin.Train Epoch: [5] Loss: 2.9364 ACC: 66.92: 100%|██████████| 40/40 [00:03<00:00, 12.61it/s]
Lin.Test Epoch: [5] Loss: 2.5228 ACC@1: 68.65% ACC@5: 94.80% : 100%|██████████| 4/4 [00:01<00:

In [14]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 5).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, task_sep=True) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 19.6145 ACC: 32.85: 100%|██████████| 196/196 [00:11<00:00, 17.14it/s]
Lin.Test Epoch: [1] Loss: 11.2666 ACC@1: 31.88% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.60it/s]
Lin.Train Epoch: [2] Loss: 13.6645 ACC: 34.04: 100%|██████████| 196/196 [00:12<00:00, 16.28it/s]
Lin.Test Epoch: [2] Loss: 23.2060 ACC@1: 31.85% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.11it/s]
Lin.Train Epoch: [3] Loss: 15.4488 ACC: 34.28: 100%|██████████| 196/196 [00:11<00:00, 16.87it/s]
Lin.Test Epoch: [3] Loss: 12.5737 ACC@1: 37.14% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.11it/s]
Lin.Train Epoch: [4] Loss: 14.7058 ACC: 34.94: 100%|██████████| 196/196 [00:11<00:00, 16.50it/s]
Lin.Test Epoch: [4] Loss: 12.3701 ACC@1: 34.22% ACC@5: 100.00% : 100%|██████████| 20/20 [00:02<00:00,  6.83it/s]
Lin.Train Epoch: [5] Loss: 14.1288 ACC: 34.57: 100%|██████████| 196/196 [00:10<00:00, 18.33it/s]
Lin.Test Epoch: [5] Loss: 8.6935 ACC@1: 38.25% ACC@5: 100.00% :

Total Accuracy: 49.66


### Task 3 Model

In [15]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard, appr_name = args.appr)
    model = Siamese(encoder)
    model.to(device) #automatically detects from model

#load model here
file_name = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_3_same_lr_True_norm_batch_ws_False.pth.tar"
dict = torch.load(file_name)
model.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)
model.load_state_dict(dict['state_dict'])

cuda:7


<All keys matched successfully>

In [16]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 100).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 4.7375 ACC: 32.44: 100%|██████████| 196/196 [00:12<00:00, 16.26it/s]
Lin.Test Epoch: [1] Loss: 4.3636 ACC@1: 39.66% ACC@5: 67.57% : 100%|██████████| 20/20 [00:03<00:00,  5.99it/s]
Lin.Train Epoch: [2] Loss: 4.8266 ACC: 37.81: 100%|██████████| 196/196 [00:12<00:00, 16.12it/s]
Lin.Test Epoch: [2] Loss: 3.9243 ACC@1: 43.78% ACC@5: 72.55% : 100%|██████████| 20/20 [00:03<00:00,  6.50it/s]
Lin.Train Epoch: [3] Loss: 4.6347 ACC: 39.58: 100%|██████████| 196/196 [00:10<00:00, 18.14it/s]
Lin.Test Epoch: [3] Loss: 4.1968 ACC@1: 42.72% ACC@5: 72.06% : 100%|██████████| 20/20 [00:03<00:00,  6.56it/s]
Lin.Train Epoch: [4] Loss: 4.7352 ACC: 40.27: 100%|██████████| 196/196 [00:11<00:00, 16.55it/s]
Lin.Test Epoch: [4] Loss: 4.0966 ACC@1: 45.22% ACC@5: 73.16% : 100%|██████████| 20/20 [00:03<00:00,  6.17it/s]
Lin.Train Epoch: [5] Loss: 4.7445 ACC: 40.91: 100%|██████████| 196/196 [00:11<00:00, 16.44it/s]
Lin.Test Epoch: [5] Loss: 3.9220 ACC@1: 45.61% ACC@5: 74.09% : 100%|████████

Total Accuracy: 58.199999999999996


In [17]:
task = 3
loader = train_data_loaders_linear[task]
lin_epoch=50
classifier = LinearClassifier(num_classes = 20).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, loader,test_data_loaders[task],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, intra_task=True) 

Lin.Train Epoch: [1] Loss: 6.0279 ACC: 51.37: 100%|██████████| 40/40 [00:03<00:00, 12.55it/s]
Lin.Test Epoch: [1] Loss: 3.4600 ACC@1: 63.60% ACC@5: 92.65% : 100%|██████████| 4/4 [00:01<00:00,  2.46it/s]
Lin.Train Epoch: [2] Loss: 3.6914 ACC: 63.98: 100%|██████████| 40/40 [00:03<00:00, 11.71it/s]
Lin.Test Epoch: [2] Loss: 2.9710 ACC@1: 68.35% ACC@5: 91.65% : 100%|██████████| 4/4 [00:01<00:00,  2.63it/s]
Lin.Train Epoch: [3] Loss: 3.1837 ACC: 65.93: 100%|██████████| 40/40 [00:03<00:00, 12.10it/s]
Lin.Test Epoch: [3] Loss: 4.2047 ACC@1: 58.95% ACC@5: 91.55% : 100%|██████████| 4/4 [00:01<00:00,  2.77it/s]
Lin.Train Epoch: [4] Loss: 3.7033 ACC: 63.77: 100%|██████████| 40/40 [00:03<00:00, 11.55it/s]
Lin.Test Epoch: [4] Loss: 3.2560 ACC@1: 66.20% ACC@5: 91.75% : 100%|██████████| 4/4 [00:01<00:00,  2.57it/s]
Lin.Train Epoch: [5] Loss: 3.2896 ACC: 67.11: 100%|██████████| 40/40 [00:03<00:00, 12.63it/s]
Lin.Test Epoch: [5] Loss: 3.2992 ACC@1: 65.80% ACC@5: 92.20% : 100%|██████████| 4/4 [00:01<00:

In [18]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 5).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, task_sep=True) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 20.4138 ACC: 33.16: 100%|██████████| 196/196 [00:11<00:00, 17.28it/s]
Lin.Test Epoch: [1] Loss: 9.0461 ACC@1: 37.48% ACC@5: 100.00% : 100%|██████████| 20/20 [00:02<00:00,  6.96it/s]
Lin.Train Epoch: [2] Loss: 15.1191 ACC: 34.58: 100%|██████████| 196/196 [00:11<00:00, 17.09it/s]
Lin.Test Epoch: [2] Loss: 14.2498 ACC@1: 38.10% ACC@5: 100.00% : 100%|██████████| 20/20 [00:02<00:00,  7.44it/s]
Lin.Train Epoch: [3] Loss: 14.7818 ACC: 34.83: 100%|██████████| 196/196 [00:09<00:00, 19.63it/s]
Lin.Test Epoch: [3] Loss: 19.0577 ACC@1: 28.55% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  5.86it/s]
Lin.Train Epoch: [4] Loss: 14.4764 ACC: 35.13: 100%|██████████| 196/196 [00:12<00:00, 16.16it/s]
Lin.Test Epoch: [4] Loss: 16.7608 ACC@1: 36.36% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.19it/s]
Lin.Train Epoch: [5] Loss: 14.7387 ACC: 34.87: 100%|██████████| 196/196 [00:11<00:00, 16.81it/s]
Lin.Test Epoch: [5] Loss: 23.6264 ACC@1: 34.47% ACC@5: 100.00% :

Total Accuracy: 49.84


### Task 4 Model

In [19]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard, appr_name = args.appr)
    model = Siamese(encoder)
    model.to(device) #automatically detects from model

#load model here
file_name = "./checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_4_same_lr_True_norm_batch_ws_False.pth.tar"
dict = torch.load(file_name)
model.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)
model.load_state_dict(dict['state_dict'])

cuda:7


<All keys matched successfully>

In [20]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 100).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 4.9028 ACC: 33.63: 100%|██████████| 196/196 [00:10<00:00, 17.91it/s]
Lin.Test Epoch: [1] Loss: 4.3413 ACC@1: 39.14% ACC@5: 70.15% : 100%|██████████| 20/20 [00:03<00:00,  6.42it/s]
Lin.Train Epoch: [2] Loss: 4.9588 ACC: 39.27: 100%|██████████| 196/196 [00:11<00:00, 16.44it/s]
Lin.Test Epoch: [2] Loss: 4.2961 ACC@1: 44.67% ACC@5: 74.41% : 100%|██████████| 20/20 [00:03<00:00,  6.21it/s]
Lin.Train Epoch: [3] Loss: 4.9856 ACC: 40.82: 100%|██████████| 196/196 [00:12<00:00, 16.29it/s]
Lin.Test Epoch: [3] Loss: 4.2225 ACC@1: 43.76% ACC@5: 72.90% : 100%|██████████| 20/20 [00:03<00:00,  6.53it/s]
Lin.Train Epoch: [4] Loss: 4.9013 ACC: 41.87: 100%|██████████| 196/196 [00:11<00:00, 16.40it/s]
Lin.Test Epoch: [4] Loss: 4.2681 ACC@1: 45.43% ACC@5: 74.65% : 100%|██████████| 20/20 [00:03<00:00,  6.31it/s]
Lin.Train Epoch: [5] Loss: 4.9165 ACC: 42.38: 100%|██████████| 196/196 [00:11<00:00, 16.48it/s]
Lin.Test Epoch: [5] Loss: 4.3893 ACC@1: 45.81% ACC@5: 74.36% : 100%|████████

Total Accuracy: 59.48


In [21]:
task = 4
loader = train_data_loaders_linear[task]
lin_epoch=50
classifier = LinearClassifier(num_classes = 20).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, loader,test_data_loaders[task],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, intra_task=True) 

Lin.Train Epoch: [1] Loss: 5.9123 ACC: 58.04: 100%|██████████| 40/40 [00:03<00:00, 12.36it/s]
Lin.Test Epoch: [1] Loss: 3.8863 ACC@1: 64.05% ACC@5: 95.60% : 100%|██████████| 4/4 [00:01<00:00,  2.61it/s]
Lin.Train Epoch: [2] Loss: 3.1740 ACC: 70.98: 100%|██████████| 40/40 [00:03<00:00, 11.75it/s]
Lin.Test Epoch: [2] Loss: 2.9595 ACC@1: 72.70% ACC@5: 93.30% : 100%|██████████| 4/4 [00:01<00:00,  2.38it/s]
Lin.Train Epoch: [3] Loss: 2.8423 ACC: 72.68: 100%|██████████| 40/40 [00:03<00:00, 12.86it/s]
Lin.Test Epoch: [3] Loss: 2.1669 ACC@1: 75.45% ACC@5: 94.50% : 100%|██████████| 4/4 [00:01<00:00,  2.43it/s]
Lin.Train Epoch: [4] Loss: 2.6900 ACC: 73.03: 100%|██████████| 40/40 [00:03<00:00, 11.96it/s]
Lin.Test Epoch: [4] Loss: 2.3647 ACC@1: 73.35% ACC@5: 95.45% : 100%|██████████| 4/4 [00:01<00:00,  2.35it/s]
Lin.Train Epoch: [5] Loss: 3.3385 ACC: 70.56: 100%|██████████| 40/40 [00:03<00:00, 11.61it/s]
Lin.Test Epoch: [5] Loss: 3.2825 ACC@1: 70.70% ACC@5: 95.85% : 100%|██████████| 4/4 [00:01<00:

In [22]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 5).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, task_sep=True) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 23.0857 ACC: 32.85: 100%|██████████| 196/196 [00:11<00:00, 16.49it/s]
Lin.Test Epoch: [1] Loss: 9.2288 ACC@1: 41.70% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.33it/s]
Lin.Train Epoch: [2] Loss: 14.9575 ACC: 34.82: 100%|██████████| 196/196 [00:12<00:00, 16.27it/s]
Lin.Test Epoch: [2] Loss: 11.4116 ACC@1: 39.16% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.62it/s]
Lin.Train Epoch: [3] Loss: 15.8021 ACC: 34.79: 100%|██████████| 196/196 [00:10<00:00, 18.21it/s]
Lin.Test Epoch: [3] Loss: 20.5715 ACC@1: 32.68% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.39it/s]
Lin.Train Epoch: [4] Loss: 15.9690 ACC: 35.01: 100%|██████████| 196/196 [00:11<00:00, 16.69it/s]
Lin.Test Epoch: [4] Loss: 11.7595 ACC@1: 38.73% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.01it/s]
Lin.Train Epoch: [5] Loss: 15.2034 ACC: 35.28: 100%|██████████| 196/196 [00:11<00:00, 16.66it/s]
Lin.Test Epoch: [5] Loss: 11.6067 ACC@1: 37.83% ACC@5: 100.00% :

Total Accuracy: 49.82


### Offline Model

In [23]:
#load model here
file_name = 'checkpoints/checkpoint_cifar100-algobasic_barlow-e[1000]-b256-lr0.3-CS[100]acc_69.38.pth.tar'
dict = torch.load(file_name)
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = 'batch', weight_standard = args.weight_standard,appr_name =args.appr)
    old_model = Siamese(encoder)
    old_model.to(device) #automatically detects from model

old_model.load_state_dict(dict['state_dict'])

cuda:7


<All keys matched successfully>

In [24]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 100).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(old_model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 2.0356 ACC: 54.21: 100%|██████████| 196/196 [00:11<00:00, 16.41it/s]
Lin.Test Epoch: [1] Loss: 1.4987 ACC@1: 62.32% ACC@5: 87.00% : 100%|██████████| 20/20 [00:03<00:00,  6.08it/s]
Lin.Train Epoch: [2] Loss: 1.8855 ACC: 58.55: 100%|██████████| 196/196 [00:11<00:00, 16.68it/s]
Lin.Test Epoch: [2] Loss: 1.5397 ACC@1: 62.54% ACC@5: 86.74% : 100%|██████████| 20/20 [00:03<00:00,  5.60it/s]
Lin.Train Epoch: [3] Loss: 1.8202 ACC: 59.74: 100%|██████████| 196/196 [00:10<00:00, 19.16it/s]
Lin.Test Epoch: [3] Loss: 1.5946 ACC@1: 62.17% ACC@5: 86.59% : 100%|██████████| 20/20 [00:02<00:00,  7.54it/s]
Lin.Train Epoch: [4] Loss: 1.8295 ACC: 60.03: 100%|██████████| 196/196 [00:12<00:00, 16.26it/s]
Lin.Test Epoch: [4] Loss: 1.5218 ACC@1: 63.38% ACC@5: 86.75% : 100%|██████████| 20/20 [00:03<00:00,  6.03it/s]
Lin.Train Epoch: [5] Loss: 1.7900 ACC: 60.58: 100%|██████████| 196/196 [00:11<00:00, 16.97it/s]
Lin.Test Epoch: [5] Loss: 1.5707 ACC@1: 62.92% ACC@5: 87.19% : 100%|████████

Total Accuracy: 69.58


In [26]:
for task, loader in enumerate(train_data_loaders_linear):
    lin_epoch=50
    classifier = LinearClassifier(num_classes = 20).to(device)
    lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
    lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
    test_loss, test_acc1, test_acc5, classifier = linear_evaluation(old_model, loader,test_data_loaders[task],lin_optimizer, classifier, 
                                                                lin_scheduler, epochs=lin_epoch, device=device, intra_task=True) 

Lin.Train Epoch: [1] Loss: 1.2729 ACC: 68.29: 100%|██████████| 40/40 [00:03<00:00, 13.07it/s]
Lin.Test Epoch: [1] Loss: 1.1626 ACC@1: 75.15% ACC@5: 96.25% : 100%|██████████| 4/4 [00:01<00:00,  2.90it/s]
Lin.Train Epoch: [2] Loss: 1.2962 ACC: 73.80: 100%|██████████| 40/40 [00:03<00:00, 12.08it/s]
Lin.Test Epoch: [2] Loss: 0.8407 ACC@1: 78.90% ACC@5: 96.95% : 100%|██████████| 4/4 [00:01<00:00,  2.95it/s]
Lin.Train Epoch: [3] Loss: 1.2123 ACC: 74.46: 100%|██████████| 40/40 [00:03<00:00, 11.85it/s]
Lin.Test Epoch: [3] Loss: 0.8776 ACC@1: 78.70% ACC@5: 97.30% : 100%|██████████| 4/4 [00:01<00:00,  2.35it/s]
Lin.Train Epoch: [4] Loss: 1.0842 ACC: 75.99: 100%|██████████| 40/40 [00:03<00:00, 12.74it/s]
Lin.Test Epoch: [4] Loss: 1.0087 ACC@1: 77.80% ACC@5: 97.35% : 100%|██████████| 4/4 [00:01<00:00,  2.71it/s]
Lin.Train Epoch: [5] Loss: 1.2006 ACC: 75.48: 100%|██████████| 40/40 [00:03<00:00, 12.04it/s]
Lin.Test Epoch: [5] Loss: 0.8478 ACC@1: 78.20% ACC@5: 97.65% : 100%|██████████| 4/4 [00:01<00:

In [25]:
lin_epoch=100
classifier = LinearClassifier(num_classes = 5).to(device)
lin_optimizer = torch.optim.SGD(classifier.parameters(), 0.2, momentum=0.9, weight_decay=0) # Infomax: no weight decay, epoch 100, cosine scheduler
lin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(lin_optimizer, lin_epoch, eta_min=0.002) #scheduler + values ref: infomax paper
test_loss, test_acc1, test_acc5, classifier = linear_evaluation(old_model, train_data_loaders_linear_all[0],
                                                            test_data_loaders_all[0],lin_optimizer, classifier, 
                                                            lin_scheduler, epochs=lin_epoch, device=device, task_sep=True) 
print(f'Total Accuracy: {test_acc1}')

Lin.Train Epoch: [1] Loss: 4.1220 ACC: 50.22: 100%|██████████| 196/196 [00:11<00:00, 16.74it/s]
Lin.Test Epoch: [1] Loss: 2.1891 ACC@1: 54.62% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.46it/s]
Lin.Train Epoch: [2] Loss: 3.1510 ACC: 51.72: 100%|██████████| 196/196 [00:11<00:00, 16.44it/s]
Lin.Test Epoch: [2] Loss: 5.3764 ACC@1: 46.30% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.50it/s]
Lin.Train Epoch: [3] Loss: 3.2916 ACC: 51.78: 100%|██████████| 196/196 [00:11<00:00, 16.38it/s]
Lin.Test Epoch: [3] Loss: 2.1499 ACC@1: 56.09% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.36it/s]
Lin.Train Epoch: [4] Loss: 2.9241 ACC: 52.10: 100%|██████████| 196/196 [00:12<00:00, 16.14it/s]
Lin.Test Epoch: [4] Loss: 3.1233 ACC@1: 50.33% ACC@5: 100.00% : 100%|██████████| 20/20 [00:03<00:00,  6.22it/s]
Lin.Train Epoch: [5] Loss: 3.1320 ACC: 51.70: 100%|██████████| 196/196 [00:10<00:00, 18.23it/s]
Lin.Test Epoch: [5] Loss: 2.4845 ACC@1: 54.45% ACC@5: 100.00% : 100%|███

Total Accuracy: 65.05
